#### Instalando e Importando Extenções

In [10]:
#!pip install azure-identity
#!pip install azure-storage-blob
#!pip install azure-keyvault-secrets
#!pip install pandas
#!pip install matplotlib
#!pip install sqlalchemy
#!pip install pyodbc

from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from azure.keyvault.secrets import SecretClient
from sqlalchemy import create_engine
from io import BytesIO

import pandas as pd
import matplotlib.pyplot as plt
import urllib.parse
import sqlalchemy

#### Selecionando o Arquivo do Container

In [6]:
KEY_VAULT_URL = "https://kv-academy-01.vault.azure.net/"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KEY_VAULT_URL, credential=credential)
credential = DefaultAzureCredential()
 
# Criar o cliente do Blob Service
blob_service_client = BlobServiceClient(account_url="https://daniel.santos.blob.core.windows.net", credential=credential)

# Nome do segredo que queremos pegar
secret_blob_name = 'stg-academy-nome'

# Pegando o segredo correspodente
retrieved_blob_secret = client.get_secret(secret_blob_name)

# Função da documentação
def get_blob_service_client_token_credential():
    #montando a url usando o segredo, note o .value para pegar o valor e nao o objeto
    account_url = "https://" + retrieved_blob_secret.value + ".blob.core.windows.net"
    #validando a credencial
    credential = DefaultAzureCredential()
    #Criando o objeto BlobServiceClient
    blob_service_client = BlobServiceClient(account_url, credential=credential)
    #retornando o objeto
    return blob_service_client

# Instanciando um objeto usando a função
blob_service_client = get_blob_service_client_token_credential()

# Uma ação qualquer para mostrar que a conexão foi feita
for container in blob_service_client.list_containers():
    print(container.name)

academy-teste
azure-webjobs-hosts
azure-webjobs-secrets
hands-on
projeto-daniel-danillo-matheus
projeto-gustavo-jonnathan
projeto-luca-aislan
projeto-matheus-thiago
projeto-michelle-thiago
yelpinc


In [20]:
# Utilizando uma função para buscar as credencias, através das keyvault, e retorna-las para acessar os bancos 
def retornaCredenciais(secret_name):
    KEY_VAULT_URL = "https://kv-academy-01.vault.azure.net/"
    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KEY_VAULT_URL, credential=credential)
 
    retrieved_secret = client.get_secret(secret_name)
    return retrieved_secret.value
 
server = retornaCredenciais("db-academy-server")
database = retornaCredenciais("db-academy-database")
username = retornaCredenciais("db-academy-user")
password = retornaCredenciais("db-academy-password")
driver= '{ODBC Driver 18 for SQL Server}'

server = server
database = database
username = username
password = password
username = urllib.parse.quote_plus(username)

# String de conexão com o SQL Server usando SQLAlchemy e pyodbc
connection_string = rf'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server'
# Criando a engine de conexão
engine = sqlalchemy.create_engine(connection_string)

In [25]:
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
    connection.close()
except Exception as e:
    print(f"Erro ao conectar: {e}")


Conexão bem-sucedida!


#### Transformação

In [12]:
# Acessando os containers e lendo o Arquivo CSV , sem baixar na máquina
container_name = 'projeto-daniel-danillo-matheus'
nome_csv = 'transactions_train.csv'

# Obter o cliente do blob e baixar os dados
blob_client = blob_service_client.get_blob_client(container=container_name, blob=nome_csv)
blob_data = blob_client.download_blob().readall()

# Ler o arquivo CSV diretamente em um DataFrame
df = pd.read_csv(BytesIO(blob_data))


In [13]:
df.head(5)

,step,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [14]:
# Dicionário para renomear as colunas
colunas_renomeadas = {
    'step': 'Tempo',
    'type': 'Tipo',
    'amount': 'Valor',
    'nameOrig': 'ContaOrigem',
    'oldbalanceOrig': 'SaldoAntigoOrigem',
    'newbalanceOrig': 'SaldoNovoOrigem',
    'nameDest': 'ContaDestino',
    'oldbalanceDest': 'SaldoAntigoDestino',
    'newbalanceDest': 'SaldoNovoDestino',
    'isFraud': 'Fraude'
}

# Aplicar a renomeação no DataFrame
df = df.rename(columns=colunas_renomeadas)

df.head()

,Tempo,Tipo,Valor,ContaOrigem,SaldoAntigoOrigem,SaldoNovoOrigem,ContaDestino,SaldoAntigoDestino,SaldoNovoDestino,Fraude
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [15]:
# Dicionário de tradução de valores
traducao_tipo = {
    'PAYMENT': 'Pagamento',
    'TRANSFER': 'Transferência',
    'CASH_OUT': 'Saque',
    'CASH_IN': 'Depósito',
    'DEBIT': 'Débito',
    'MONEY_TRANSFER': 'Transferência de Dinheiro'
}

# Substituir os valores na coluna 'Tipo' conforme o dicionário
df['Tipo'] = df['Tipo'].replace(traducao_tipo)

df.head()

,Tempo,Tipo,Valor,ContaOrigem,SaldoAntigoOrigem,SaldoNovoOrigem,ContaDestino,SaldoAntigoDestino,SaldoNovoDestino,Fraude
0,1,Pagamento,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,Pagamento,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,Transferência,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,Saque,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,Pagamento,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6351193 entries, 0 to 6351192
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Tempo               int64  
 1   Tipo                object 
 2   Valor               float64
 3   ContaOrigem         object 
 4   SaldoAntigoOrigem   float64
 5   SaldoNovoOrigem     float64
 6   ContaDestino        object 
 7   SaldoAntigoDestino  float64
 8   SaldoNovoDestino    float64
 9   Fraude              int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 484.6+ MB


In [17]:
# Verificando a quantidade de valores nulos por coluna
nulos_por_coluna = df.isnull().sum()

print("Quantidade de valores nulos por coluna:")
print(nulos_por_coluna)

# Para verificar se há ao menos um valor nulo no DataFrame
tem_nulos = df.isnull().values.any()
print(f"\nO DataFrame contém valores nulos? {'Sim' if tem_nulos else 'Não'}")


Quantidade de valores nulos por coluna:
Tempo                 0
Tipo                  0
Valor                 0
ContaOrigem           0
SaldoAntigoOrigem     0
SaldoNovoOrigem       0
ContaDestino          0
SaldoAntigoDestino    0
SaldoNovoDestino      0
Fraude                0
dtype: int64

O DataFrame contém valores nulos? Não


#### Carregar para o Banco

In [26]:
# Inserindo os dados no SQL Server
df.to_sql('projeto_daniel_danillo_matheus', con=engine, if_exists='replace', index=False)

101